In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = pd.read_csv('google_playstore_cleaned_trail.csv')

In [3]:
print(dataset.columns)
dataset = dataset.drop(['App Name'], axis=1)
print(dataset.columns)

Index(['App Name', 'Category', 'Rating', 'Rating Count', 'Installs', 'Price',
       'Size', 'Content Rating', 'Game_genre'],
      dtype='object')
Index(['Category', 'Rating', 'Rating Count', 'Installs', 'Price', 'Size',
       'Content Rating', 'Game_genre'],
      dtype='object')


In [4]:
dataset.loc[dataset['Installs'].isin(['0 - 100', '100 - 500', '500 - 1,000']), 'Installs'] = '0 - 1,000'
dataset.loc[dataset['Installs'].isin(['1,000 - 5,000', '5,000 - 10,000']), 'Installs'] = '1,000 - 10,000'
dataset.loc[dataset['Installs'].isin(['10,000 - 50,000', '50,000 - 100,000']), 'Installs'] = '10,000 - 100,000'
dataset.loc[dataset['Installs'].isin(['100,000 - 500,000', '500,000 - 1,000,000']), 'Installs'] = '100,000 - 1,000,000'
dataset.loc[dataset['Installs'].isin(['1,000,000 - 5,000,000', '5,000,000 - 10,000,000']), 'Installs'] = '1,000,000 - 10,000,000'
dataset.loc[dataset['Installs'].isin(
    ['10,000,000 - 50,000,000', '50,000,000 - 100,000,000']), 'Installs'] = '10,000,000 - 100,000,000'
dataset.loc[dataset['Installs'].isin(
    ['100,000,000 - 500,000,000', '500,000,000 - 1,000,000,000']), 'Installs'] = '100,000,000 - 1,000,000,000'
dataset.loc[dataset['Installs'].isin(['1,000,000,000 - 5,000,000,000', '5,000,000,000+']), 'Installs'] = '1,000,000,000+'

dataset.Installs = pd.Categorical(dataset.Installs, ['0 - 1,000', '1,000 - 10,000', '10,000 - 100,000', '100,000 - 1,000,000',
                                           '1,000,000 - 10,000,000', '10,000,000 - 100,000,000',
                                           '100,000,000 - 1,000,000,000', '1,000,000,000+'])

In [5]:
print(dataset.Installs.value_counts().sort_index())
print(dataset.shape)

0 - 1,000                      650975
1,000 - 10,000                 306211
10,000 - 100,000               198642
100,000 - 1,000,000             92666
1,000,000 - 10,000,000          31807
10,000,000 - 100,000,000         6306
100,000,000 - 1,000,000,000       534
1,000,000,000+                     49
Name: Installs, dtype: int64
(1287191, 8)


In [6]:
strat = dataset.Category.values
dataset = pd.get_dummies(dataset, columns=['Content Rating', 'Category', 'Game_genre'], drop_first=True)
print(dataset.columns)

Index(['Rating', 'Rating Count', 'Installs', 'Price', 'Size',
       'Content Rating_Everyone', 'Content Rating_Everyone 10+',
       'Content Rating_Mature 17+', 'Content Rating_Teen',
       'Content Rating_Unrated', 'Category_Adventure', 'Category_Arcade',
       'Category_Art & Design', 'Category_Auto & Vehicles', 'Category_Beauty',
       'Category_Board', 'Category_Books & Reference', 'Category_Business',
       'Category_Card', 'Category_Casino', 'Category_Casual',
       'Category_Comics', 'Category_Communication', 'Category_Dating',
       'Category_Education', 'Category_Educational', 'Category_Entertainment',
       'Category_Events', 'Category_Finance', 'Category_Food & Drink',
       'Category_Health & Fitness', 'Category_House & Home',
       'Category_Libraries & Demo', 'Category_Lifestyle',
       'Category_Maps & Navigation', 'Category_Medical', 'Category_Music',
       'Category_Music & Audio', 'Category_News & Magazines',
       'Category_Parenting', 'Category_Persona

In [7]:
X = dataset.drop(['Installs'], axis=1).values
y = dataset.Installs.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=strat, random_state=42)

In [8]:
y_test=y_test.astype('category')
y_train=y_train.astype('category')
y=y.astype('category')

In [10]:
dt = DecisionTreeClassifier(max_depth=8, max_features='sqrt')
dt.fit(X_train, y_train)
y_pred = dt.predict(X_train)

acc = accuracy_score(y_train, y_pred)
print("Decision Tree train data accuracy: {:.2f}".format(acc))

Decision Tree train data accuracy: 0.61


In [11]:
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

acc = accuracy_score(y_test.astype(str), y_pred)
print("Decision Tree Test data accuracy: {:.2f}".format(acc))

Decision Tree Test data accuracy: 0.60


In [13]:
rf = RandomForestClassifier(max_depth=20, n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_train)

acc = accuracy_score(y_train, y_pred)
print("Random Forest Train data accuracy: {:.2f}".format(acc))

Random Forest Train data accuracy: 0.82


In [14]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

acc = accuracy_score(y_test.astype(str), y_pred)
print("Random Forest Test data accuracy: {:.5f}".format(acc))

Random Forest Test data accuracy: 0.79237


In [ ]:
gbm = GradientBoostingClassifier(learning_rate=0.05, max_depth=8, n_estimators=400, verbose=1)
gbm.fit(X_train, y_train)

y_pred = gbm.predict(X_test)

acc = accuracy_score(y_test.astype(str), y_pred)
print("Gradient Boosting Classifier Test data accuracy: {:.5f}".format(acc))

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('knn train data accuracy', knn.score(X_train, y_train))
print('knn test data accuracy', knn.score(X_test, y_test.astype(str)))
print('knn cross validation accuracy', np.mean(cross_val_score(knn, X, y, cv=5)))